In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import math
import json
import random
import csv
import multiprocessing
import sys
import pickle
import os
import gc
from annoy import AnnoyIndex
import nmslib
#import tensorflow as tf
import bottleneck as bn

import heapq
import time
import operator
from collections import Counter,defaultdict

from lightfm import LightFM
from lightfm.evaluation import auc_score,precision_at_k,recall_at_k
from lightfm.data import Dataset
from lightfm.cross_validation import random_train_test_split

from sklearn.feature_selection import chi2, SelectKBest
from sklearn.preprocessing import normalize
from sklearn.metrics import accuracy_score,confusion_matrix, roc_curve, auc,classification_report
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix, coo_matrix, hstack,dok_matrix, lil_matrix, vstack

mpl.rcParams['figure.dpi']= 100

DATAPATH="/mnt/projects/continuous_scanning/trackingChange/recData/"
NUM_THREADS=72

from model import extractData, modelPrep, misc
from bench import main 


#mpl.rcParams['figure.dpi']= 150

In [3]:
print("True Retrain, UNIFORM Sample  and Oversample0.64 Train,slash16,more filtered dataset, retrain all false, EF_Construction 1K,10kCalc Weights")


BENCH2_CYCLES = 500
BIASED=True

RETRAIN = True
RETRAIN_WEIGHTS = False
WITH_ITEM_FEATS=False
BIASED = False
HIST_BINS=100
EF_CONSTRUCTION=1000#1000
M = 20

START = 0
LIM = 4539900
NUM_PREDS = 500#300

RETRAIN_GAP = 1
RETRAIN_ALL = False
RETRAIN_EPOCHS =10
      
CALC_WEIGHTS = True

#'''
curFeat = "asn"
f = DATAPATH + "ht65k_1pset_uniform200_l10.csv"
df, dataset, num_users, num_items, interactions, weights, \
user_features,userf_train, item_features, train,test,trainw,testw,dmap, inv_ip_map, \
inv_port_map = extractData.extractData(f,curFeat, train_percentage = 0.1, \
                           withItemFeats=WITH_ITEM_FEATS,calcWeights= CALC_WEIGHTS,\
                          weightCap = 10000)



hybrid_model = LightFM(loss='warp', #'warp
    random_state=2016,
    learning_rate=0.01, #0.05
    no_components= 4000,#4000,
    #learning_schedule='adadelta',
    max_sampled= 175,
    #user_alpha=0.5

    )


print("Initialized Model")
hybrid_model = hybrid_model.fit(train, #newtrain
                  user_features=userf_train,
                  #item_features=item_features,
                  sample_weight = trainw,
                  epochs=80, #20
                  num_threads=NUM_THREADS, verbose=True)




num_test_ips, num_services,num_ips_per_port, test_rows, testcoo, predictions, \
    testPorts,LIM = \
    modelPrep.prepTheModel(hybrid_model,test,train,LIM,START,HIST_BINS,user_features,\
                 item_features,BIASED = BIASED,WITH_ITEM_FEATS=WITH_ITEM_FEATS,\
                 NUM_PREDS=NUM_PREDS,EF_CONSTRUCTION =EF_CONSTRUCTION, NUM_THREADS=NUM_THREADS )

#'''
model_description = misc.constructDescript("0.1Train_uniform40_PlayingRetrainGap10_4KComp_0.01LR_RetrainAllFalse_EF1K_10kCalcWeightsTrue_ASN_", RETRAIN =RETRAIN, RETRAIN_WEIGHTS =  RETRAIN_WEIGHTS, BIASED=BIASED)
print(model_description)
bench2_results,predictions,all_correctly_predicted = main.bench2_Improvements(hybrid_model,predictions,num_services,num_test_ips,\
                                           test,testw,test_rows,train,user_features,item_features, \
                                           inv_port_map, dmap, num_ips_per_port,\
                                           NUM_THREADS,NUM_PREDS, RETRAIN,RETRAIN_GAP, RETRAIN_ALL,RETRAIN_EPOCHS,BIASED, LIM,HIST_BINS, BENCH2_CYCLES,\
                                           RETRAIN_WEIGHTS,CALC_WEIGHTS,WITH_ITEM_FEATS,EF_CONSTRUCTION,M,model_description, DATAPATH)














True Retrain, UNIFORM Sample  and Oversample0.64 Train,slash16,more filtered dataset, retrain all false, EF_Construction 1K,10kCalc Weights
Num users: 482028, num_items 64061.
<482028x64061 sparse matrix of type '<class 'numpy.int32'>'
	with 737585 stored elements in COOrdinate format>
Initialized Model


Epoch: 100%|██████████| 80/80 [07:07<00:00,  5.34s/it]


...Grabbed Representations
Got Item Representations
Got User Representations
Made Index
Got Top Items
Got Predictions/Top Items
...Calculated Predictions
0.1Train_uniform40_PlayingRetrainGap10_4KComp_0.01LR_RetrainAllFalse_EF1K_10kCalcWeightsTrue_ASN_0.01LR_4000NCOMP_10EPOCHS_175MAXSAMPLE_FalseBIASED_TrueRETRAIN_FalseRETRAINWEIGHTS_FalseWITHITEMFEATS_FalseCALCWEIGHTS
-------
...Beginning Bench2: Compare Different Metrics of Coverage and Hitrate
cycle:  0


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

correct: 22731
Num Scanned:  433826
Number unique ports Rec'd:  8894
At Bin:  0
Hitrate:  0.052396582961832625
total ips correct:  22731
Total Normalized Services:  362.80197980848556
Total Normalized IPs:  865.6278746788381
Total Fraction Services: 0.03425381062529668
Total Fraction IPs: 0.052396582961832625
Total Fraction Depopularized Services: 0.03425373528324798
Total Fraction Depopularized IPs: 0.052424530654083056


...

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Total Normalized Services:  3520.9378308191835
Total Normalized IPs:  8171.182322269715
Total Fraction Services: 0.27627730351639906
Total Fraction IPs: 0.33078699755201396
Total Fraction Depopularized Services: 0.2759802997277259
Total Fraction Depopularized IPs: 0.33041675516442115


Epoch: 100%|██████████| 10/10 [00:33<00:00,  3.39s/it]


...Grabbed Representations
Got Item Representations
Got User Representations
Made Index
Got Top Items
Got Predictions/Top Items
...Calculated Predictions
...calculations complete


NameError: name 'pd' is not defined